In [1]:
import datetime
from pathlib import Path

import glob2
import numpy as np
import scipy
from tqdm import tqdm

In [2]:
paths = ["/media/plerolland/LaBoite/IMS/2014/H01W1/20140101.w"]
TO_VOLT = 5.0 / 2 ** 24
SENSIBILITY = -163.5

In [4]:
for path in tqdm(paths):
    dir = "/".join(path.split("/")[:-1])
    Path(f'{dir}/atomic').mkdir(parents=True, exist_ok=True)
    fname = path.split("/")[-1][:-2]
    path_wfdisc = f'{dir}/wfdisc/{fname}.wfdisc'
    with open(path, 'rb') as file:
        file.seek(0)
        data = np.fromfile(file, dtype=">i4")
    data = scipy.signal.detrend(data) * TO_VOLT / 10 ** (SENSIBILITY / 20)
    with open(path_wfdisc, 'rb') as file:
        header = file.readlines()
    for i in range(len(header)):
        line = header[i].split()
        start = datetime.datetime.utcfromtimestamp(float(line[2]))
        end = datetime.datetime.utcfromtimestamp(float(line[6]))
        start_data_idx = int(line[17]) // 4
        data_to_save = data[start_data_idx:]
        if i < len(header) - 1:
            end_data_idx = int(header[i+1][17]) // 4
            data_to_save = data[:end_data_idx-start_data_idx]
        file_to_save = f'{start.strftime("%Y%m%d_%H%M%S")}_-_{end.strftime("%Y%m%d_%H%M%S")}'
        file_to_save = f'{dir}/atomic/{file_to_save}.w'
        if not Path(file_to_save).exists():
            with open(file_to_save, 'wb') as f:
                data_to_save.tofile(f)

100%|██████████| 64/64 [00:00<00:00, 35320.45it/s]


In [6]:
from matplotlib import pyplot as plt
import datetime
from pathlib import Path
import glob2
import numpy as np
import scipy
from tqdm import tqdm

paths = glob2.glob("/media/plerolland/LaBoite/IMS/*/*/*.w")
for path in tqdm(paths):
    already_touched = False
    _dir = "/".join(path.split("/")[:-1])
    atomic_dir = f"{_dir}/atomic/"
    Path(atomic_dir).mkdir(exist_ok=True)
    already_computed = glob2.glob(f"{atomic_dir}/*.w")
    if len(already_computed) > 0:
        last = already_computed[-1]
        end_date = last.split("_")[-2] + "_" + last.split("_")[-1][:-2]
        end_date = datetime.datetime.strptime(end_date, "%Y%m%d_%H%M%S")
        already_touched = True
    
    fname = path.split("/")[-1][:-2]
    path_wfdisc = f'{_dir}/wfdisc/{fname}.wfdisc'
    with open(path_wfdisc, 'rb') as file:
        header = file.readlines()
    last_date = datetime.datetime.utcfromtimestamp(float(header[-1].split()[6]))
    if already_touched and last_date < end_date:
        continue   # .w file already treated
    
    with open(path, 'rb') as file:
        file.seek(0)
        data = np.fromfile(file, dtype=">f4")
    data = (data - 120000)*float(header[-1].split()[9])
    for i in tqdm(range(len(header)), leave=False):
        line = header[i].split()
        start = datetime.datetime.utcfromtimestamp(float(line[2]))
        end = datetime.datetime.utcfromtimestamp(float(line[6]))
        start_data_idx = int(line[17]) // 4
        data_to_save = data[start_data_idx:]
        if i < len(header) - 1:
            end_data_idx = int(header[i+1].split()[17]) // 4
            data_to_save = data[:end_data_idx-start_data_idx]
        file_to_save = f'{start.strftime("%Y%m%d_%H%M%S")}_-_{end.strftime("%Y%m%d_%H%M%S")}'
        file_to_save = f'{atomic_dir}/{file_to_save}.wav'
        if not Path(file_to_save).exists():
            scipy.io.wavfile.write(file_to_save, np.rint(float(line[8])), data_to_save)

  0%|          | 1/221 [01:07<4:08:13, 67.70s/it]


KeyboardInterrupt: 